In [15]:
import click
import requests
import pandas as pd


df = pd.read_csv('./journal_abbreviations_acs.csv',delimiter=';',header=None)
df.rename(columns={0: "long_name", 1: "short_name"},inplace=True)

def get_json(doi):
    url = f'http://api.crossref.org/works/{doi}'
    resp = requests.get(url)
    found = False if resp.status_code != 200 else True
    entry_json = resp.json()
    return found, entry_json

def find_validate(doi):
    found = False
    try:
        found, entry = get_json(doi)
    except:
        raise ValueError('Could not perform the request!')
    
    if found:
        return entry

def synthesize_roam_import(doi):
    
    json = find_validate(doi)
#     title
    t = json['message']['title'][0]
    title = f'- {t}\n'
#     Abstract
    if 'abstract' in json['message'].keys():
        abst = json['message']['abstract'][0]
        abstract = f'- Abstract:\n\t- {abst}'
    else:
        abstract = f'- Abstract:\n\t- \n'
    
#     References - Authors
    ref_base = '- Reference:\n'
    auth_base = '\t- Authors:\n'
    
    author_list = json['message']['author']
    a_list = []
    for index,a in enumerate(author_list):
        if index == 0:
            filename_author = a['family']
        given = a['given']
        family = a['family']
        item = f'[[{given} {family}]]'
        auth_base += '\t\t- '+item +'\n'

    ref = ref_base + auth_base 
#     Reference - Journal
    journal = json['message']['container-title']
    abbr = df[df['long_name'] == journal[0]]['short_name'].values[0]
    filename_journal = abbr.replace(' ','').replace('.','')

    journal = f'\t- Journal:\n\t\t- [[{journal[0]}]] [[{abbr}]]\n'
#     Url
    if 'url' in json['message'].keys():
        u = json['message']['url']
    elif 'URL' in json['message'].keys():
        u = json['message']['URL']
    url = f'\t- URL:\n\t\t- {u}\n'
    
    empty_fields = '- Notes:\n- Summary:\n- Concepts:\n- Keywords:\n'
    
    filename_year = json['message']['created']['date-parts'][0][0]
    
    filename = f'{filename_author}EtAl_{filename_journal}{filename_year}'
    
    entry = title + abstract + ref + journal + url + empty_fields
    
    print(filename)
    print(entry)
    
    return filename, entry
    

        

In [17]:
synthesize_roam_import(doi)

Monzón-GonzálezEtAl_JPhysChemSolids2020
- Synthesis and characterization of organotin(IV) semiconductors and their applications in optoelectronics
- Abstract:
	- 
- Reference:
	- Authors:
		- [[César R. Monzón-González]]
		- [[María Elena Sánchez-Vergara]]
		- [[Wilmer E. Vallejo Narváez]]
		- [[Tomás Rocha-Rinza]]
		- [[Marcos Hernández]]
		- [[Elizabeth Gómez]]
		- [[Omar Jiménez-Sandoval]]
		- [[Cecilio Álvarez-Toledano]]
	- Journal:
		- [[Journal of Physics and Chemistry of Solids]] [[J. Phys. Chem. Solids]]
	- URL:
		- http://dx.doi.org/10.1016/j.jpcs.2020.109840
- Notes:
- Summary:
- Concepts:
- Keywords:



('Monzón-GonzálezEtAl_JPhysChemSolids2020',
 '- Synthesis and characterization of organotin(IV) semiconductors and their applications in optoelectronics\n- Abstract:\n\t- \n- Reference:\n\t- Authors:\n\t\t- [[César R. Monzón-González]]\n\t\t- [[María Elena Sánchez-Vergara]]\n\t\t- [[Wilmer E. Vallejo Narváez]]\n\t\t- [[Tomás Rocha-Rinza]]\n\t\t- [[Marcos Hernández]]\n\t\t- [[Elizabeth Gómez]]\n\t\t- [[Omar Jiménez-Sandoval]]\n\t\t- [[Cecilio Álvarez-Toledano]]\n\t- Journal:\n\t\t- [[Journal of Physics and Chemistry of Solids]] [[J. Phys. Chem. Solids]]\n\t- URL:\n\t\t- http://dx.doi.org/10.1016/j.jpcs.2020.109840\n- Notes:\n- Summary:\n- Concepts:\n- Keywords:\n')

In [19]:
# doi = '10.1021/acsami.8b15684'
# doi = '10.1021/acsami.8b04600'
doi = '10.1016/j.jpcs.2020.109840'
doi = '10.1002/anie.202007983'

In [18]:
df

,long_name,short_name
0,ACS Applied Materials & Interfaces,ACS Appl. Mater. Interfaces
1,ACS Applied Nano Materials,ACS Appl. Nano Mater.
2,ACS Biomaterials Science & Engineering,ACS Biomater. Sci. Eng.
3,ACS Catalysis,ACS Catal.
4,ACS Central Science,ACS Cent. Sci.
...,...,...
1734,"Zeitschrift fuer Naturforschung, C: Journal of...","Z. Naturforsch., C: J. Biosci."
1735,Zeitschrift fuer Physikalische Chemie (Muenche...,"Z. Phys. Chem. (Muenchen, Ger.)"
1736,Zhipu Xuebao,Zhipu Xuebao
1737,Zhongguo Shengwu Huaxue Yu Fenzi Shengwu Xuebao,Zhongguo Shengwu Huaxue Yu Fenzi Shengwu Xuebao


In [26]:
found, bib = get_bib(doi)

In [32]:
bib

"@article{Monz_n_Gonz_lez_2021,\n\tdoi = {10.1016/j.jpcs.2020.109840},\n\turl = {https://doi.org/10.1016%2Fj.jpcs.2020.109840},\n\tyear = 2021,\n\tmonth = {mar},\n\tpublisher = {Elsevier {BV}},\n\tvolume = {150},\n\tpages = {109840},\n\tauthor = {C{\\'{e}}sar R. Monz{\\'{o}}n-Gonz{\\'{a}}lez and Mar{\\'{\\i}}a Elena S{\\'{a}}nchez-Vergara and Wilmer E. Vallejo Narv{\\'{a}}ez and Tom{\\'{a}}s Rocha-Rinza and Marcos Hern{\\'{a}}ndez and Elizabeth G{\\'{o}}mez and Omar Jim{\\'{e}}nez-Sandoval and Cecilio {\\'{A}}lvarez-Toledano},\n\ttitle = {Synthesis and characterization of organotin({IV}) semiconductors and their applications in optoelectronics},\n\tjournal = {Journal of Physics and Chemistry of Solids}\n}"

In [20]:
found, js = get_json(doi)

In [22]:
js['message']

{'indexed': {'date-parts': [[2021, 2, 24]],
  'date-time': '2021-02-24T17:47:06Z',
  'timestamp': 1614188826920},
 'reference-count': 49,
 'publisher': 'Wiley',
 'issue': '43',
 'license': [{'URL': 'http://onlinelibrary.wiley.com/termsAndConditions#vor',
   'start': {'date-parts': [[2020, 8, 25]],
    'date-time': '2020-08-25T00:00:00Z',
    'timestamp': 1598313600000},
   'delay-in-days': 0,
   'content-version': 'vor'},
  {'URL': 'http://doi.wiley.com/10.1002/tdm_license_1.1',
   'start': {'date-parts': [[2020, 8, 25]],
    'date-time': '2020-08-25T00:00:00Z',
    'timestamp': 1598313600000},
   'delay-in-days': 0,
   'content-version': 'tdm'}],
 'funder': [{'DOI': '10.13039/501100001665',
   'name': 'Agence Nationale de la Recherche',
   'doi-asserted-by': 'publisher',
   'award': ['ANR-11-IDEX-0004-02']}],
 'content-domain': {'domain': ['onlinelibrary.wiley.com'],
  'crossmark-restriction': True},
 'short-container-title': ['Angew. Chem. Int. Ed.'],
 'published-print': {'date-parts

In [21]:
tsts = synthesize_md_string(js)

IndexError: index 0 is out of bounds for axis 0 with size 0

In [41]:
tsts[1]

'- Synthesis and characterization of organotin(IV) semiconductors and their applications in optoelectronics\n- Abstract:\n\t- \n- Reference:\n\t- Authors:\n\t\t- [[César R. Monzón-González]]\n\t\t- [[María Elena Sánchez-Vergara]]\n\t\t- [[Wilmer E. Vallejo Narváez]]\n\t\t- [[Tomás Rocha-Rinza]]\n\t\t- [[Marcos Hernández]]\n\t\t- [[Elizabeth Gómez]]\n\t\t- [[Omar Jiménez-Sandoval]]\n\t\t- [[Cecilio Álvarez-Toledano]]\n\t- Journal:\n\t\t- [[Journal of Physics and Chemistry of Solids]] [[J. Phys. Chem. Solids]]\n\t- URL:\n\t\t- http://dx.doi.org/10.1016/j.jpcs.2020.109840\n- Notes:\n- Summary:\n- Concepts:\n- Keywords:\n'

In [43]:
with open(f'./{tsts[0]}.md','w') as fout:
    fout.write(tsts[1])

In [119]:
with open('./pa.md', 'r') as handler:
    ff = handler.read()